# Getting started with PyQuil

## Get API key

Go to http://forest.rigetti.com/ and request a Forest API key.
You will receive an e-mail containing your API key and user ID.

## Set up the config

Copy the key and enter them in the field below.
After the key and user ID have been filled out, evaluate the cell with SHIFT + ENTER. This will create the config file which will be used to connect to the Rigetti Forest API.

In [ ]:
%%writefile .pyquil_config
[Rigetti Forest]
key: 
user_id: 

## Imports

Now first import the dependencies. Evaluate these cells with SHIFT+ENTER.

In [9]:
from pyquil.api import QPUConnection, QVMConnection, get_devices
qvm = QVMConnection()

In [10]:
from pyquil.quil import Program
from pyquil.gates import X, MEASURE

## Create a program & run it on the Quantum Virtual Machine (QVM)

Let's create our first program! This program runs an X pulse (or a "NOT" gate) and measures it. The result is stored in classical address 0.

In [ ]:
program = Program()
program += X(0)
program += MEASURE(0, 0)

In [ ]:
print(program)

Now run the program on the QVM!

In [ ]:
qvm.run(program, classical_addresses=[0], trials=10)

## Entanglement

Let's entangle two qubits!

In [ ]:
from pyquil.gates import H, CNOT

In [ ]:
program = Program()
program += H(0)
program += CNOT(0, 1)
program += MEASURE(0, 0)
program += MEASURE(1, 1)

In [ ]:
single_shot_data = qvm.run(program, classical_addresses=[0, 1], trials=10)
single_shot_data

### Entanglement using native CZ gate

Our natural gateset uses a CZ instead of a CNOT. One can transform a CNOT into CZs and thereby run the entangling operation as follows:

In [ ]:
from pyquil.gates import CZ

program = Program()
program += H(0)
program += H(1)
program += CZ(0, 1)
program += H(1)
program += MEASURE(0, 0)
program += MEASURE(1, 1)

qvm.run(program, classical_addresses=[0, 1], trials=10)

## Plot the parity

Evaluate the following lines to plot the results of the single shot data for the entangling operation.

In [ ]:
%pylab inline
parity = {
    (0, 0): 0,
    (0, 1): 1,
    (1, 0): 2,
    (1, 1): 3
}

In [ ]:
def plot_parity(single_shot_data):
    plt.hist([parity[(u, v)] for (u, v) in single_shot_data])
    plt.xticks(list(parity.values()), parity.keys())
    plt.title("Bell test results.")
    ;

In [ ]:
plot_parity(single_shot_data)

### Run on the QPU

In [24]:
get_devices()

/Users/guen/env/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
return the following:

    get_devices()
    # {'19Q-Acorn': <Device 19Q-Acorn online>, '8Q-Agave': <Device 8Q-Agave offline>}
    acorn = get_devices()['19Q-Acorn']

To use this Dict return type now, you may optionally pass the flag get_devices(as_dict=True). This
will become the default behavior in a future pyQuil release.

  """Entry point for launching an IPython kernel.


{<Device 19Q-Acorn online>, <Device 8Q-Agave online>}

If you have received QPU access, you can run these programs on the QPU! For that, we need to connect to the QPU first.

In [35]:
qpu = QPUConnection("8Q-Agave")

In [36]:
program = Program()
program += X(0)

In [37]:
qpu.run_and_measure(program, [0], trials=10)

InvalidInputError: Error: missing or invalid device.

The server returned the above error because something was wrong with the HTTP
request sent to it. This could be due to a bug in the server or a bug in your
code. If you suspect this to be a bug in pyQuil or Rigetti Forest, then please
describe the problem in a GitHub issue at:
    https://github.com/rigetticomputing/pyquil/issues